# Guided Project - Answering Business Questions Using SQL
### By Steven Paravati

In this project I will be working with a modified version of a database called Chinook. The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.

In [1]:
# Connecting to the chinook database:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [3]:
# taking a look at the tables in the chinook database:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


To get some practice and explore the data further, first I will just take a look at the tracks sold by genre, including the quantity, min, max and average unit price and then figuring out the average purchase totals by genre. It will be ordered by total quantity sold descending from the highest to lowest amount.  

In [33]:
%%sql
SELECT 
    g.name,
    COUNT(il.quantity) AS genre_sold,
    MIN(il.unit_price) AS minimum_price,
    MAX(il.unit_price) AS maximum_price,
    ROUND(AVG(il.unit_price), 2) AS average_price,
    ROUND((SUM(il.quantity) * (AVG(il.unit_price))), 2) AS average_purchases
FROM invoice_line il
INNER JOIN track t ON il.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1 ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


name,genre_sold,minimum_price,maximum_price,average_price,average_purchases
Rock,2635,0.99,0.99,0.99,2608.65
Metal,619,0.99,0.99,0.99,612.81
Alternative & Punk,492,0.99,0.99,0.99,487.08
Latin,167,0.99,0.99,0.99,165.33
R&B/Soul,159,0.99,0.99,0.99,157.41
Blues,124,0.99,0.99,0.99,122.76
Jazz,121,0.99,0.99,0.99,119.79
Alternative,117,0.99,0.99,0.99,115.83
Easy Listening,74,0.99,0.99,0.99,73.26
Pop,63,0.99,0.99,0.99,62.37


In [91]:
%%sql
SELECT 
    g.name,
    COUNT(il.quantity) AS genre_sold,
    ROUND((cast(count(il.quantity) AS FLOAT)/(
                                               SELECT COUNT(*)
                                               FROM invoice_line il
                                               ) * 100)
    , 2) percent_of_total
FROM invoice_line il
INNER JOIN track t ON il.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1 ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


name,genre_sold,percent_of_total
Rock,2635,55.39
Metal,619,13.01
Alternative & Punk,492,10.34
Latin,167,3.51
R&B/Soul,159,3.34
Blues,124,2.61
Jazz,121,2.54
Alternative,117,2.46
Easy Listening,74,1.56
Pop,63,1.32


Even if the pricing is the same for all tracks and makes this table a little underwheming, it's still good information to know that, for this store's data, the pricing will be the same. We can also see the biggest selling genres here are rock related: Rock, Metal, Alternative & Punk make up 80% of the total sales around the world. 

## Chinook signs a deal
The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:


| Artist Name | Genre |
| :-: | :-: |
|Regal |Hip-Hop |
|Red Tone |Punk |
|Meteor and the Girls | Pop |
|Slim Jim Bites | Blues |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

I will need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store. It works out that my exploration of the tables gave me a head start, so now we will update our initial query to focus just on the USA:

In [94]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    ROUND((cast(count(uts.invoice_line_id) AS FLOAT) / (
                                                        SELECT COUNT(*) 
                                                        FROM usa_tracks_sold
                                                      ) * 100)
    , 2) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


The genres appear roughly the same distribution in the USA as it was worldwide. Hip-Hop/Rap is an extremely popular genre of music, but just for *streaming*, not for album sales. Based on these results I would suggest to the label we just work the Red Tone punk album, and promote it with in-person events where customers can meet the band, through online promotions targeted to those areas, or a mixture of both. 

In [133]:
%%sql

SELECT 
    c.state,
    SUM(il.quantity) total_sold
    FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    INNER JOIN track t on t.track_id = il.track_id
    INNER JOIN genre g on g.genre_id = t.genre_id    
    WHERE c.country = "USA" and g.name = "Alternative & Punk"
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 5;

 * sqlite:///chinook.db
Done.


state,total_sold
CA,26
WI,18
WA,18
FL,17
NY,15


The top 5 states where we sold the most alternative/punk albums are California, Wisconsin, Washington, Florida and New York. We should coordinate with the label to promote the album more heavily in these locations.

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. This section will analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [137]:
%%sql

WITH customer_support_rep_sales AS
    (
     SELECT
         i.customer_id,
         c.support_rep_id,
         SUM(i.total) total
     FROM invoice i
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY 1,2
    )

SELECT
    e.first_name || " " || e.last_name employee,
    e.hire_date,
    "$" || ROUND(SUM(csrs.total), 2) total_sales
FROM customer_support_rep_sales csrs
INNER JOIN employee e ON e.employee_id = csrs.support_rep_id
GROUP BY 1;

 * sqlite:///chinook.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,$1731.51
Margaret Park,2017-05-03 00:00:00,$1584.0
Steve Johnson,2017-10-17 00:00:00,$1393.92


In [141]:
%%sql

SELECT MIN(invoice_date) AS first_invoice from invoice;

 * sqlite:///chinook.db
Done.


first_invoice
2017-01-03 00:00:00


Based on the results here, there isn't too much at issue with any of the employee's performances. The employee with the least amount of total sales was also the one that was hired last. It may even make the case that Steve's performance is the best for being only 20% below the top employee's sales despite being hired several months later.

## Analyzing Sales by Country

Your next task is to analyze the sales data for customers from each different country. For this exercise I will use use the country value from the `customers` table, and ignore the country from the billing address in the `invoice` table.

In particular, I will calculate data for each country on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [144]:

%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    num_of_customers,
    total_sales,
    avg_order_sales,
    avg_customer_sales
FROM
    (
    SELECT
        country,
        count(distinct customer_id) num_of_customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) avg_customer_sales,
        SUM(unit_price) / count(distinct invoice_id) avg_order_sales,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, avg_customer_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,num_of_customers,total_sales,avg_order_sales,avg_customer_sales
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


Based on the results, there may be opportunity in the following countries:

- Czech Republic
- United Kingdom
- India

However, the actual number of customers in those countries is extremely low, so any effort to increase sales in those countries should be focused on driving new customers to the store, not providing incentives to the small amount of existing customers.

## Albums vs Individual Tracks Comparison
The Chinook store is setup in a way that allows customer to make purchases in one of two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

Our task is to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [149]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    was_album_purchased,
    COUNT(invoice_id) number_of_invoices,
    ROUND(CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) *100
    , 2) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "was_album_purchased"
     FROM invoice_first_track ifs
    )
GROUP BY was_album_purchased;

 * sqlite:///chinook.db
Done.


was_album_purchased,number_of_invoices,percent
no,500,81.43
yes,114,18.57


Given that album purchases make up nearly 20% of all invoices, it does not appear to be a good idea to drop the ability to purchase albums from the store. Instead, management should work with the developers to incentivize customers to purchase the full album if they have bought tracks from it. The ability to complete an album purchase has worked well for other online track sales stores like iTunes, and could drive revenue from the other direction.